In [ ]:
# 准备 fast align 输入语料
import sys

dir = "/Users/lisan/spec/spec.fast_align.input"

ofile = open(dir + "spec.fast_align.input", "w")

ja_file = dir + "train.tok.clean.ja"
cn_file = dir + "train.tok.clean.cn"
ja_lines = [line.strip() for line in open(ja_file).readlines()]
cn_lines = [line.strip() for line in open(cn_file).readlines()]
for (j,c) in zip(ja_lines, cn_lines):
    ofile.write(j + "|||" + c + "\n")
    
ja_file = dir + "test.tok.ja"
cn_file = dir + "test.tok.cn"
ja_lines = [line.strip() for line in open(ja_file).readlines()]
cn_lines = [line.strip() for line in open(cn_file).readlines()]
for (j,c) in zip(ja_lines, cn_lines):
    ofile.write(j + "|||" + c + "\n")
    
print("Done!")


In [ ]:

# 运行 fast align

./fast_align -i spec.fast_align.input -d -o -v > forward.align
./fast_align -i spec.fast_align.input -d -o -v -r > reverse.align
./atools -i forward.align -j reverse.align -c grow-diag-final-and > bidrction.align


In [4]:
# fast align 输出转化成 soft alignment 的 attention 格式

import sys

dir = "/Users/lisan/spec/spec.fast_align.input"

corpus = open(dir + "spec.fast_align.input").readlines()
align = open(dir + "bidirection.align").readlines()

ofile = open(dir + "soft_alignment.input", "w")

ja_lines = [line.strip().split("|||")[0] for line in corpus]
cn_lines = [line.strip().split("|||")[1] for line in corpus]
align_lines = [line.strip() for line in align]
for (ja,cn,a) in zip(ja_lines, cn_lines, align_lines): 
    J = len(cn.strip().split(" "))
    I = len(ja.strip().split(" ")) 
    ofile.write("0 ||| %s ||| 0 ||| %s ||| %s %s\n" % (ja,cn, str(I), str(J)))
    
    ss = a.split(" ")
    attn = {}
    for s in ss:
        i, _, j = s.rpartition("-")
        i, j = int(i), int(j)
        if j not in attn.keys():
            attn[j] = [i]
        else:
            attn[j].append(i)
    
    
    for j in range(J):
        scores = [0.0] * I
        
        ii = attn.get(j, [])
        score = 0.0
        if len(ii) > 0:
            score = 1.0 / len(ii)
            
        ii = set(ii)    
        for i in range(I):
            if i in ii:
                scores[i] = score
        scores = [str(x) for x in scores]
        ofile.write(" ".join(scores) + "\n")
    ofile.write("\n")
    


FileNotFoundError: [Errno 2] No such file or directory: '/Users/lisan/spec/spec.fast_align.inputspec.fast_align.input'